
# Implementación del Patrón Factory Method: Acceso a recursos HTTP o FTP 

En este ejemplo vamos a crear una herramienta que permita acceder a recursos web usando **protocolo HTTP o FTP**, aplicando el patrón de diseño **Factory Method**.

---

### Contexto del problema:

- Algunos servidores web permiten acceder a sus recursos vía **FTP** (usando un cliente tipo FileZilla).
- Otros permiten acceder a los mismos recursos vía **HTTP o HTTPS** (desde un navegador).

Por ejemplo, el servidor `ftp.freebsd.org` es accesible tanto por **http://ftp.freebsd.org** como por **ftp://ftp.freebsd.org**.

---

### Objetivo del ejercicio:

Diseñar una aplicación en Python que pueda:

- Obtener un listado de archivos desde un servidor.
- Elegir en tiempo de ejecución entre **HTTP** o **FTP**.
- Usar un **mismo cliente** pero con **conectores distintos**, gracias a la aplicación del patrón **Factory Method**.

Cada tipo de conexión (HTTP o FTP) se implementará como una **subclase concreta**, que decide:

- Qué protocolo utilizar.
- Qué puerto utilizar.
- Cómo interpretar (parsear) la respuesta del servidor.

---

**Tecnologías utilizadas:**

- `requests` → Para conexiones HTTP/HTTPS.
- `ftplib` → Para conexiones FTP.
- `BeautifulSoup` → Para parseo de HTML.
- `abc` → Para definir clases abstractas.



In [2]:

from abc import ABC, abstractmethod
import requests
from requests.exceptions import RequestException
from ftplib import FTP
from bs4 import BeautifulSoup
from typing import List


## Productos concretos: puertos

In [3]:

class Port(ABC):
    @abstractmethod
    def __str__(self): ...

class HTTPPort(Port):
    def __str__(self): return "80"

class HTTPSecurePort(Port):
    def __str__(self): return "443"

class FTPPort(Port):
    def __str__(self): return "21"


## Connector

In [5]:

class Connector(ABC):
    """Define el Factory Method (`port_factory_method`) y la lógica común."""
    def __init__(self, secure: bool):
        self.secure = secure                # HTTPS vs HTTP
        self.port = self.port_factory_method()
        self.protocol = self.protocol_factory_method()

    @abstractmethod
    def protocol_factory_method(self) -> str:
        pass

    @abstractmethod
    def port_factory_method(self) -> Port:
        pass

    @abstractmethod
    def read(self, host: str, path: str) -> bytes | None:
        """Lee bytes del recurso remoto (puede devolver None si falla)."""

    @abstractmethod
    def parse_listing(self, raw: bytes) -> List[str]:
        """Extrae lista de archivos del contenido descargado."""


## Creator concreto: HTTPConnector

In [6]:

class HTTPConnector(Connector):
    def protocol_factory_method(self) -> str:
        return "https" if self.secure else "http"

    def port_factory_method(self) -> Port:
        return HTTPSecurePort() if self.secure else HTTPPort()

    def read(self, host: str, path: str) -> bytes | None:
        url = f"{self.protocol}://{host}:{self.port}{path}"
        print("GET", url)
        try:
            resp = requests.get(url, timeout=5)
            resp.raise_for_status()
            return resp.content
        except RequestException as e:
            print("[HTTP] Error:", e)
            return None

    def parse_listing(self, raw: bytes) -> List[str]:
        """Parsea un <table> HTML típica de un index de Apache/NGINX."""
        soup = BeautifulSoup(raw, 'html.parser')
        out = []
        if soup.table:
            for link in soup.table.find_all('a'):
                out.append(link.get_text(strip=True))
        return out


## Creator concreto: FTPConnector

In [7]:

class FTPConnector(Connector):
    def protocol_factory_method(self) -> str:
        return "ftp"

    def port_factory_method(self) -> Port:
        return FTPPort()

    def read(self, host: str, path: str) -> bytes | None:
        """Devuelve el listado de directorio como bytes (texto)."""
        print(f"Conectando vía FTP a {host}:{self.port}{path}")
        try:
            with FTP() as ftp:
                ftp.connect(host, int(str(self.port)), timeout=5)
                ftp.login()                       # acceso anónimo
                ftp.cwd(path)
                lines = []
                ftp.retrlines('LIST', callback=lines.append)
                return "\n".join(lines).encode()
        except Exception as e:
            print("[FTP] Error:", e)
            return None

    def parse_listing(self, raw: bytes) -> List[str]:
        lines = raw.decode(errors='ignore').split('\n')
        files = []
        for line in lines:
            parts = line.split(None, 8)
            if len(parts) == 9:
                files.append(parts[-1])
        return files


## Cliente interactivo

In [9]:

domain = 'ftp.gnu.org'
path = '/'  # directorio raíz

proto = int(input("Protocolo (0=HTTP, 1=FTP): "))

if proto == 0:
    secure = bool(int(input("¿HTTPS? (1=Sí,0=No): ")))
    connector = HTTPConnector(secure)
else:
    connector = FTPConnector(False)

raw = connector.read(domain, path)
if raw:
    files = connector.parse_listing(raw)
    print("\nArchivos encontrados:")
    for f in files:
        print(" -", f)
else:
    print("No se pudo obtener el contenido.")


GET https://ftp.gnu.org:443/

Archivos encontrados:
 - Name
 - Last modified
 - Size
 - Description
 - CRYPTO.README
 - MISSING-FILES
 - MISSING-FILES.README
 - README
 - before-2003-08-01.md5sums.asc
 - find.txt.gz
 - gnu+linux-distros/
 - gnu/
 - ls-lrRt.txt.gz
 - mirrors/
 - non-gnu/
 - old-gnu/
 - pub/
 - robots.txt
 - savannah/
 - third-party/
 - tmp/
 - tree.json.gz
 - video/
 - welcome.msg


# Resumen

En este notebook hemos implementado el patrón **Factory Method** para desacoplar:

1. **La lógica de negocio común** (lectura de contenido remoto)  
de  
2. **La lógica de creación de objetos específicos** (elección de protocolo y puerto).

---

### Ventajas de este enfoque:

- Podemos **cambiar o ampliar** el número de protocolos en el futuro sin modificar el código cliente.
- El cliente solo conoce la **interfaz común (Connector)**, no las subclases concretas.
- Mantenemos **bajo acoplamiento** y favorecemos el principio **Open/Closed**.

---


Este es un ejemplo práctico de cómo el patrón **Factory Method** nos ayuda a crear código extensible y mantenible en aplicaciones reales.
